<a href="https://colab.research.google.com/github/asminimulin/cv-course/blob/main/01-image-classification/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the data

You can download CIFAR10 dataset from official site:

https://www.cs.toronto.edu/~kriz/cifar.html

or do it using Pytorch:
https://pytorch.org/vision/stable/datasets.html#cifar

or another framework.



In [ ]:
# Put your code here

classes = ['Самолет', 'Автомобиль', 'Птица', 'Кошка', 'Олень', 'Собака', 'Лягушка', 'Лошадь', 'Корабль', 'Грузовик']

## Split data into train and validation parts

Extract subset containing train and validation parts. 
Part must contain 5000 and 500 samples respectively.

In [ ]:
# Put your code here


# Train data 5000 samples
X_train =  # ...
y_train =  # ...

# Validation data 500 samples
X_val = # ...
y_val = # ...

## Dispaly some images  along with it class names
  


In [ ]:
# Put your code here

# Implement NearestNeighbor class

In [ ]:
class NearestNeighbor:
  def __init__(self): # You can change the method signature
    pass

  def fit(self,x,y): # You can change the method signature
    """
      Arguments:
        x  (Tensor or numpy.array): collection of objects from testset (batch)
        y  (Tensor or numpy.array): collection of integer 
        representing a class number for objects from x
    
    """
    pass
  
  def predict(self,x):
    """
      Arguments:
          x  (Tensor or numpy.array): collection of objects from testset (batch)
      
      Returns:
          class_num (Tensor or numpy.array) - collection of integer representing
          class numbers for objects from x             
    """
    
    # ... Put you code here

    return labels

##Perform smoke test

- Create model instance
- get predictions for dozen of samples


In [ ]:
model_nn = NearestNeighbor()
# Put your code here if need 
model.nn.fit(X_train)
predicted_labels = model.predict(X_test[0:9])
for i, label in enumerate(predicted_labels):
  print(classes[label],"vs",classes[y_test[i]])

# Validate your model

## Create validation function

Must calculate Accuracy metric for your model.

In [ ]:
def validate(model,x, y):
  """
      Arguments:
          model (Object): instance of NearestNeighbor class
          x (Tensor or numpy.array): collection of objects 
          y (Tensor or numpy.array): collection of integer representing
          class numbers for objects from x   
      
      Returns:
          accuracy (float) : percentage of correctly classified objects 
    """
  
  # ... Put you code here
  
  return accuracy

 ## Calculate model accuracy on validation data

In [ ]:
accuracy = validate(model_nn,X_test,y_test) 
print ("Accuracy {:.3f}".format(accuracy)

# Place for brief conclusion
Feel free to describe troubles here.


...



# Ideas for extra work

- Implenment K-NearestNeighbor
- Test different distance functions
- Find hyperparams using Cross-Validation
- Plot accuracy curve against K
- Evaluate time  